In [12]:
import os
import pandas as pd

In [13]:
#os.system("kaggle competitions download -c blg454e-summer-2021")

In [16]:
sample_submission_loc = "blg454e-summer-2021/sampleSubmission.csv"
test0_loc = "blg454e-summer-2021/test_t0.csv"
train0_loc = "blg454e-summer-2021/train_t0.csv"
train1_loc = "blg454e-summer-2021/train_t1.csv"

In [17]:
test0_data = pd.read_csv(test0_loc)
train0_data = pd.read_csv(train0_loc)
train1_data = pd.read_csv(train1_loc)